# Better way 32. 지연 속성에는 `__getattr__`, `__getattribute__`, `__setattr__` 을 사용하자

* 파이썬의 **language hook** 를 이용하면 시스템들을 연계하는 범용 코드를 쉽게 만들 수 있다.
  * 예) 데이터베이스의 row 를 파이썬 객체로 표현한다고 하자.
    * 데이터베이스에는 schema set 이 존재함
    * 그러므로 row 를 사용하는 코드는 데이터베이스의 형태를 알아야 함
    * 하지만 **코드를 범용으로 만들면** 객체와 데이터베이스를 연결하는 코드에서 row 의 schema 를 몰라도 됨
  * 여기서 `__getattr__`, `__getattribute__`, `__setattr__` = hook

* 일반 인스턴스 속성, @property, 디스크립터로는 불가능하지만 `__getattr__` 메서드를 override 하면 가능

In [1]:
class LazyDB(object):
    def __init__(self):
        self.exists = 5
        
    def __getattr__(self, name):
        value = 'Value for %s' % name
        setattr(self, name, value)
        return value

* 존재하지 않는 foo 속성에 접근해보자
  * `__getattr__` 이 호출됨 --> `__dict__` 를 변경하게 됨

In [2]:
data = LazyDB()
print('Before:', data.__dict__)
print('foo:   ', data.foo)
print('After: ', data.__dict__)

Before: {'exists': 5}
foo:    Value for foo
After:  {'exists': 5, 'foo': 'Value for foo'}


* `LazyDB` 에 로깅을 추가해보자.

In [3]:
class LoggingLazyDB(LazyDB):
    def __getattr__(self, name):
        print('Called __getattr__(%s)' % name)
        
        # 무한 반복을 피하기 위해 LazyDB 의 __getattr__ 을 호출함
        return super().__getattr__(name)

In [4]:
data = LoggingLazyDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)

exists: 5
Called __getattr__(foo)
foo:    Value for foo
foo:    Value for foo


* exists 는 이미 존재하므로 `__getattr__` 이 호출되지 않음
* foo 는 처음에는 `__getattr__` 이 호출 (`LazyDB.__getattr__` 에서 `setattr` 을 호출하여 딕셔너리에 foo 가 저장) --> 두번째에는 호출되지 않음

### Schemaless data
* schema 가 정해지지 않은 데이터를 말함
* `__getattr__` 을 이용하면 schemaless data 에 지연 접근하도록 구현할 때 큰 도움이 된다.

* 데이터베이스 시스템에서 트랜잭션 구현하기
  * 트랜잭션 설명: https://coding-factory.tistory.com/226
    * 어떤 작업 하나 혹은 여러 작업들을 하나로 묶어서, commit 으로 한꺼번에 수행하게 만들거나, rollback 하면 원래 상태로 되돌려주는 것
    * 둘 이상의 트랜잭션이 병행 수행되면 데이터가 꼬일 수 있으므로 끼어들지 않게 해야함
  * 사용자가 데이터베이스의 속성에 접근할 때 row 가 유효한지, 트랜잭션이 여전히 열려있는지 알고 싶음
    * `__getattr__`은 처음 객체를 만들때만 호출되므로 부적합함
    * `__getattribute__` 는 객체의 속성에 접근할 때마다 호출되는 hook, 트랜잭션의 상태를 확인하는 작업에 적합함

In [5]:
class ValidatingDB(object):
    def __init__(self):
        self.exists = 5
        
    def __getattribute__(self, name):
        # log 를 남김
        print('Called __getattribute__(%s)' % name)
        try:
            # 이미 존재하는 속성은 정상적으로 작동
            return super().__getattribute__(name)
        except AttributeError:
            # 존재하지 않는 속성은 지연 속성으로 만들어주기
            value = 'Value for %s' % name
            setattr(self, name, value)
            return value

In [6]:
data = ValidatingDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)

Called __getattribute__(exists)
exists: 5
Called __getattribute__(foo)
foo:    Value for foo
Called __getattribute__(foo)
foo:    Value for foo


* 만약 동적으로 접근한 프로퍼티가 **존재하지 않아야 하는 경우** 에는 파이썬 표준 동작이 일어나도록 구현하기
  * 데이터베이스 schema 에 존재하지 않는 프로퍼티 등
  * `AttributeError`

In [7]:
class MissingPropertyDB(object):
    def __init__(self):
        self.exists = 5
        
    def __getattribute__(self, name):
        if name == 'bad_name':
            raise AttributeError('%s is missing' % name)
        
        print('Called __getattribute__(%s)' % name)
        try:
            return super().__getattribute__(name)
        except AttributeError:
            value = 'Value for %s' % name
            setattr(self, name, value)
            return value

In [8]:
data = MissingPropertyDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)
print('bad name:', data.bad_name)

Called __getattribute__(exists)
exists: 5
Called __getattribute__(foo)
foo:    Value for foo
Called __getattribute__(foo)
foo:    Value for foo


AttributeError: bad_name is missing

* `hasattr` (프로퍼티가 있는지 확인), `getattr` (프로퍼티 값 가져오기)
  * 이 내장 함수들도 `__getattr__`쓰기 전에 인스턴스 딕셔너리에서 속성 이름을 찾음
  * = 존재하면 `__getattr__` 을 호출하지 않음

In [9]:
data = LoggingLazyDB()
print('Before:    ', data.__dict__)
print('foo exsits:', hasattr(data, 'foo'))
print('After:     ', data.__dict__)
print('foo exsits:', hasattr(data, 'foo'))

Before:     {'exists': 5}
Called __getattr__(foo)
foo exsits: True
After:      {'exists': 5, 'foo': 'Value for foo'}
foo exsits: True


* `__getattribute__` 를 구현하면 `hasattr`, `getattr` 호출할 때 마다 실행됨

In [10]:
data = ValidatingDB()
print('Before:    ', data.__dict__)
print('foo exsits:', hasattr(data, 'foo'))
print('After:     ', data.__dict__)
print('foo exsits:', hasattr(data, 'foo'))

Called __getattribute__(__dict__)
Before:     {'exists': 5}
Called __getattribute__(foo)
foo exsits: True
Called __getattribute__(__dict__)
After:      {'exists': 5, 'foo': 'Value for foo'}
Called __getattribute__(foo)
foo exsits: True


* 지연 방식으로 데이터를 데이터베이스에 넣기
  * `__setattr__`: 임의의 속성 할당을 가로채는 메소드.
    * 인스턴스의 속성이 할당 받을때마다 직접 혹은 내장함수 `setattr` 을 통해 호출된다.
  

In [11]:
class SavingDB(object):
    def __setattr__(self, name, value):
        # 몇몇 데이터를 DB 로그로 저장함
        # 생략
        
        super().__setattr__(name, value)

In [12]:
class LoggingSavingDB(object):
    def __setattr__(self, name, value):
        print('Called __setattr__(%s)' % name)
        super().__setattr__(name, value)

In [13]:
data = LoggingSavingDB()
print('Before: ', data.__dict__)
data.foo = 5
print('After:  ', data.__dict__)
data.foo = 7
print('Fianlly: ', data.__dict__)

Before:  {}
Called __setattr__(foo)
After:   {'foo': 5}
Called __setattr__(foo)
Fianlly:  {'foo': 7}


* `__getattribute__` 를 사용할 때 문제가 생길 수 있음
  * 객체의 속성에 접근할때마다 호출하기 때문에...

In [14]:
# 객체의 속성에 접근하면 딕셔너리에서 키를 찾게 하고 싶음
class BrokenDictionaryDB(object):
    def __init__(self, data):
        self._data = data
        
    def __getattribute__(self, name):
        #print('Called __getattribute__(%s)' % name)
        return self._data[name] # 문제의 주범

In [15]:
data = BrokenDictionaryDB({'foo': 3})
data.foo

RecursionError: maximum recursion depth exceeded

* 해결책: `super().__getattribute__('_data')` 로 재귀호출을 피하기

In [16]:
class DictionaryDB(object):
    def __init__(self, data):
        self._data = data
        
    def __getattribute__(self, name):
        data_dict = super().__getattribute__('_data')
        return data_dict[name]

In [17]:
data = DictionaryDB({'foo': 3})
data.foo

3

* `__setattr__` 에서도 같은 일이 발생할 수 있으니 `super().__setattr__(name, value)` 를 써야함
  * 여태까지 잘 쓰고 있었긴 함

In [23]:
class BrokenDB(object):
    def __init__(self):
        self.exists = 3
    
    def __setattr__(self, name, value):
        #print('Called __setattr__(%s)' % name)
        self.name = value

In [24]:
data = BrokenDB()

RecursionError: maximum recursion depth exceeded

In [21]:
class NormalDB(object):
    def __init__(self):
        self.exists = 3
    
    def __setattr__(self, name, value):
        print('Called __setattr__(%s)' % name)
        super().__setattr__(name, value)

In [22]:
data = NormalDB()
data.foo = 5
print(data.foo)

Called __setattr__(exists)
Called __setattr__(foo)
5


### 핵심 정리
* 객체의 속성을 지연 방식으로 로드하고 저장하려면 `__getattr__` 과 `__setattr__` 을 사용하자
* `__getattr__` 은 존재하지 않는 속성에 접근할 때 한 번만 호출되는 반면 `__getattribute__` 는 속성에 접근할 때 마다 호출된다는 점을 이해하자
* `__getattribute__` 와 `__setattr__` 에서 인스턴스 속성에 직접 접근할 때 super() (즉, object 클래스) 의 메서드를 사용하여 무한 재귀가 일어나지 않도록 하자